In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from torchvision import models
import torchvision.utils
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
# helper function
def imshow(img, title):
    img = torchvision.utils.make_grid(img, normalize=True)
    npimg = img.numpy()
    fig = plt.figure(figsize=(5, 15))
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.title(title)
    plt.axis("off")
    plt.show()

<h1> Prepare Data</h1>

In [ ]:
import dataset


In [ ]:
model = models.inception_v3(pretrained=True)
model.aux_logits = False

for parameter in model.parameters():
    parameter.requires_grad = False

model.fc = nn.Sequential(nn.Linear(model.fc.in_features, 10), nn.Linear(10, 2))

model = model.cuda()

loss = nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(
    filter(lambda p: p.requires_grad, model.parameters()), lr=0.001
)


num_epochs = 30

for epoch in range(num_epochs):
    total_batch = len(train_data) // batch_size

    for i, (batch_images, batch_labels) in enumerate(train_loader):
        X = batch_images.cuda()
        Y = batch_labels.cuda()

        pre = model(X)
        cost = loss(pre, Y)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        if (i + 1) % 5 == 0:
            print(
                "Epoch [%d/%d], lter [%d/%d] Loss: %.4f"
                % (epoch + 1, num_epochs, i + 1, total_batch, cost.item())
            )

In [ ]:
model.eval()

correct = 0
total = 0

for images, labels in test_loader:
    images = images.cuda()
    outputs = model(images)

    _, predicted = torch.max(outputs.data, 1)

    total += labels.size(0)
    correct += (predicted == labels.cuda()).sum()

print("Accuracy of test images: %f %%" % (100 * float(correct) / total))

classes = ["Squirrel", "Chipmunk"]

images, labels = iter(test_loader).next()

outputs = model(images.cuda())

_, predicted = torch.max(outputs.data, 1)

print("Predicted: ", " ".join("%5s" % classes[predicted[j]] for j in range(5)))

title = " ".join("%5s" % classes[labels[j]] for j in range(5))
imshow(torchvision.utils.make_grid(images, normalize=True), title)